In Switzerland, when you buy residential property, you need to fulfill two specific conditions.

- Condition 1: Equity ratio (*Belehnungsgrad*). You can take out a mortgage amounting to max. 80% of the value of the property. The remaining 20% must be available as "equity capital", of which at least 10% must be cash assets. The remaining 10% may be drawn from the pension fund (*Pensionskasse*).
- Condition 2: Affordability (*Tragbarkeit*). The running costs must not exceed 33% of your gross income.

:::{.column-page}
```{shinylive-python}
#| standalone: true
#| viewerHeight: 600

import numpy as np

from shiny import *
from shinywidgets import output_widget, register_widget
import plotly.graph_objs as go


def get_cash_assets1(price, pensionskasse):
  pensionskasse_frac = pensionskasse/price
  if pensionskasse_frac >= 0.1:
    hard_cash = price*0.1
  else:
    hard_cash = (0.2-pensionskasse_frac)*price
    
  return(hard_cash)

def get_cash_assets2(price, annual_gross_income, pensionskasse, deckungsgrad_limit = 0.33):
    hard_cash = price - pensionskasse - (deckungsgrad_limit*annual_gross_income-0.01*price)/0.06
    return(hard_cash)


def get_cash_assets(price, annual_gross_income, pensionskasse, deckungsgrad_limit = 0.33):
    hard_cash1 = get_cash_assets1(price, pensionskasse)
    hard_cash2 = get_cash_assets2(price, annual_gross_income, pensionskasse, deckungsgrad_limit = deckungsgrad_limit)
    return(max(hard_cash1, hard_cash2))

app_ui = ui.page_fluid(
    ui.row(
        ui.input_slider("p", "Price Range", value=[100000, 1000000], min=50000, max=3000000, step = 100000),

    ),
    ui.row(
        ui.input_numeric("pk","Pension Fund",100000, min = 0),
        ui.input_numeric("income","Annual gross income",130000, min = 0),
    ),
    ui.row(
        output_widget("scatterplot"),
    )
)




def server(input, output, session):

    @reactive.Calc
    def price():
        by = (input.p()[1]-input.p()[0])/100
        return([x for x in range(input.p()[0],input.p()[1],int(by))])

    @reactive.Calc
    def cash():
        cash = {str(int(perc*100)): [get_cash_assets(x, input.income(), input.pk(), perc) for x in price()] for perc in [0.33,0.36]}
        return(cash)

    scatterplot = go.FigureWidget(
        layout={"showlegend": False, "hovermode": "x unified"},
    )
    
    register_widget("scatterplot", scatterplot)

    @reactive.Effect
    def _():
        scatterplot.data = []
        for key, val in cash().items():
          scatterplot.add_scatter(x=price(), y=val, mode="lines",name = key+"%")

    




app = App(app_ui, server)
```

:::

## Condition 1: Equity ratio (*Belehnungsgrad*)

As mentioned, the bank will provide you with maximum 80% of the money needed to buy the residential property. The remaining 20% must be available as "equity capital", of which at least 10% must be cash assets. The remaining 10% may be drawn from the pension fund (*Pensionskasse*). Cash assets is money can be money on a bank deposit, in a 3rd pillar (*3a Säule*) or a gift / inheritance. 


In [ ]:
def get_cash_assets1(price, pensionskasse):
  pensionskasse_frac = pensionskasse/price
  if pensionskasse_frac >= 0.1:
    hard_cash = price*0.1
  else:
    hard_cash = (0.2-pensionskasse_frac)*price
    
  return(hard_cash)

## Condition 2: Affordability (*Tragbarkeit*)

While condition 1 limits the height of your mortgabe based on the price of the property, condition 2 is limiting based on your yearly income. This condition states that the running costs of your house should not exceed 33% of your gross income (which I will call $\gamma$). The running costs are based on the height of your mortgage (which I will call $\alpha$) and the price of your property (which I will call $\beta$):

1. The imputed interest (*Kalkulatorischer Zins*): This is not the *actual* interest on your mortgage (since this might change in the future) but a fixed interest of 5%. Formally:  $\alpha \times 5\%$.
2. Amortisation costs (*Amortisationskosten*): Traditionally, you are required to pay back your "second mortgage" (normally 15% of your full morgage) within 15 years. Formally: $\alpha \times 1\%$
3. Ancillary costs (*Nebenkosten*): These are calculated to be 1% of your house price (not your morgage). Formally: $\beta \times 1\%$

Putting it all together, condition 2 can be formulated as $\gamma \times 33\% \leq \alpha \times 5\% + \alpha \times 1\% + \beta \times 1\%$. Since I'm mostly intered in the minimal cash assets needed, I need to solve this function for $\alpha$, the mortgage:

\begin{align}
0.33\gamma &= 0.05\alpha + 0.01\alpha + 0.01\beta \\
0.33\gamma - 0.01\beta &= 0.06\alpha \\
\alpha &= \frac{0.33\gamma - 0.01\beta}{0.06} \\
\end{align}

We can further simplify $\alpha$, the mortgage, as being the price ($\beta$), minus pensionskasse ($\delta$) minus hard cash ($\epsilon$)

\begin{align}
\alpha &= \frac{0.33\gamma - 0.01\beta}{0.06} \\
\beta - \delta - \epsilon &= \frac{0.33\gamma - 0.01\beta}{0.06} \\
\epsilon &= \beta - \delta  - \frac{0.33\gamma - 0.01\beta}{0.06} \\

\end{align}


Now that we have solved it for the $\epsilon$, we can translate the formula into a python function:


In [ ]:
def get_cash_assets2(price, annual_gross_income, pensionskasse, deckungsgrad_limit = 0.33):
  hard_cash = price - pensionskasse - (deckungsgrad_limit*annual_gross_income-0.01*price)/0.06
  return(hard_cash)

## Bringing it together

Since both conditions must be met, we can combine both functions into one, so that for any given situation, we know the minimum amount of hard cash necessary. 


In [ ]:
def get_cash_assets(price, annual_gross_income, pensionskasse, deckungsgrad_limit = 0.33):
  hard_cash1 = get_cash_assets1(price, pensionskasse)
  hard_cash2 = get_cash_assets2(price, annual_gross_income, pensionskasse, deckungsgrad_limit = deckungsgrad_limit)
  return(max(hard_cash1, hard_cash2))